In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict,KFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB


In [ ]:
df=pd.read_csv('/content/train_s3TEQDk.csv')
df.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [ ]:
df2=pd.read_csv('/content/test_mSzZ8RL.csv')
df2.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No


In [ ]:
df3 = df.append(df2)

In [ ]:
df3['Credit_Product']=df3['Credit_Product'].fillna(-99)

In [ ]:
df3.isnull().sum()

Gender                                            0
Age                                               0
Region_Code                                       0
Occupation                                        0
Channel_Code                                      0
                                                 ..
city_aggregate_features5Gender_count              0
city_aggregate_features5Credit_Product_nunique    0
city_aggregate_features5Credit_Product_count      0
city_aggregate_features5Is_Active_nunique         0
city_aggregate_features5Is_Active_count           0
Length: 137, dtype: int64

In [ ]:
df3['Vintage']= df3['Vintage']/12

In [ ]:
df3['Gender']=df3['Gender'].replace({'Male':1,'Female':0})
df3['Occupation']=df3['Occupation'].replace({'Self_Employed':0,'Salaried':1,'Other':2,'Entrepreneur':3})
df3['Channel_Code']=df3['Channel_Code'].replace({'X1':1,'X3':2,'X2':3,'X4':4})
#df['Credit_Product']=df['Credit_Product'].replace({'No':1,'Yes':2})
df3['Is_Active']=df3['Is_Active'].replace({'No':0,'Yes':1})
df3['Credit_Product']=df3['Credit_Product'].replace({'No':0,'Yes':1})

#df3['Avg_Account_Balance']=np.log1p(df3['Avg_Account_Balance'])
df3['Age_Group'] = np.where((df3['Age']<30) & (df3['Age'] > 60),0,1)

In [ ]:
combine_set=df3

In [ ]:
combine_set['Total_premium_Channelwise']=combine_set.groupby(['Vintage'])['Avg_Account_Balance'].transform('sum')
combine_set['Mean_premium_Channelwise']=combine_set.groupby(['Vintage'])['Avg_Account_Balance'].transform('mean')
combine_set['Maximum_premium_Channelwise']=combine_set.groupby(['Vintage'])['Avg_Account_Balance'].transform('max')
combine_set['Min_premium_Channelwise']=combine_set.groupby(['Vintage'])['Avg_Account_Balance'].transform('min')
combine_set['Total_premium_regionwise']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].transform('sum')
combine_set['Mean_premium_regionwise']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].transform('mean')
combine_set['Max_premium_regionwise']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].transform('max')
combine_set['Min_premium_regionwise']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].transform('min')
combine_set['Age_groups_region_wise']=combine_set.groupby(['Region_Code'])['Age'].transform('nunique')
combine_set['regionwise_channels']=combine_set.groupby(['Vintage'])['Region_Code'].transform('nunique')
combine_set['Channelwise_regions']=combine_set.groupby(['Region_Code'])['Vintage'].transform('nunique')
combine_set['Unique_customers_based_Vinatge']=combine_set.groupby(['Region_Code','Gender'])['Vintage'].transform('nunique')
combine_set['Region_wise_Vehicle_Age_premium']=combine_set.groupby(['Region_Code','Occupation'])['Avg_Account_Balance'].transform('sum')
combine_set['Region_wise_Vehicle_Age_premium_mean']=combine_set.groupby(['Region_Code','Occupation'])['Avg_Account_Balance'].transform('mean')
combine_set['Region_wise_Vehicle_Age_premium_max']=combine_set.groupby(['Region_Code','Occupation'])['Avg_Account_Balance'].transform('max')
combine_set['Channel_wise_Vehicle_Age_premium']=combine_set.groupby(['Channel_Code', 'Occupation'])['Avg_Account_Balance'].transform('sum')
combine_set['Channel_wise_Vehicle_Age_premium_mean']=combine_set.groupby(['Channel_Code', 'Occupation'])['Avg_Account_Balance'].transform('mean')
combine_set['Channel_wise_Vehicle_Age_premium_max']=combine_set.groupby(['Channel_Code', 'Occupation'])['Avg_Account_Balance'].transform('max')


combine_set['Rank_regionwise_premium']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].rank(method='first',ascending=True)
combine_set['Rank_mean_regionwise_premium']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].rank(method='average',ascending=True)
combine_set['Rank_max_regionwise_premium']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].rank(method='max',ascending=True)
combine_set['Rank_min_regionwise_premium']=combine_set.groupby(['Region_Code'])['Avg_Account_Balance'].rank(method='min',ascending=True)
combine_set['Rank_regionwise_diff']=combine_set['Rank_max_regionwise_premium']- combine_set['Rank_min_regionwise_premium']
combine_set['Rank_channelwise_premium']=combine_set.groupby(['Channel_Code'])['Avg_Account_Balance'].rank(method='first',ascending=True)
combine_set['Rank_mean_channelwise_premium']=combine_set.groupby(['Channel_Code'])['Avg_Account_Balance'].rank(method='average',ascending=True)
combine_set['Rank_max_channelwise_premium']=combine_set.groupby(['Channel_Code'])['Avg_Account_Balance'].rank(method='max',ascending=True)
combine_set['Rank_min_channelwise_premium']=combine_set.groupby(['Channel_Code'])['Avg_Account_Balance'].rank(method='min',ascending=True)
combine_set['Rank_channelwise_diff']=combine_set['Rank_max_channelwise_premium']- combine_set['Rank_min_channelwise_premium']
combine_set['Rank_Channel_wise_Vehicle_Age_Premium']=combine_set.groupby(['Channel_Code','Age'])['Avg_Account_Balance'].rank(method='first',ascending=True)
combine_set['Rank_Region_wise_Vehicle_Age_premium']=combine_set.groupby(['Region_Code','Age'])['Avg_Account_Balance'].rank(method='first',ascending=True)
combine_set['Rank_Age_wise_premium']=combine_set.groupby(['Age'])['Avg_Account_Balance'].rank(method='first',ascending=True)


In [ ]:
combine_set['InsuranceLicense'] = combine_set['Is_Active'].astype('str') + '' + combine_set['Credit_Product'].astype('str')
combine_set['InsuranceGender'] = combine_set['Is_Active'].astype('str') + '' + combine_set['Occupation'].astype('str')
combine_set['InsuranceGender1'] = combine_set['Is_Active'].astype('str') + '' + combine_set['Channel_Code'].astype('str')
combine_set['InsuranceGender2'] = combine_set['Gender'].astype('str') + '' + combine_set['Credit_Product'].astype('str')
combine_set['InsuranceGender3'] = combine_set['Gender'].astype('str') + '' + combine_set['Occupation'].astype('str')
combine_set['InsuranceGender4'] = combine_set['Gender'].astype('str') + '' + combine_set['Channel_Code'].astype('str')
combine_set['InsuranceGender5'] = combine_set['Occupation'].astype('str') + '' + combine_set['Channel_Code'].astype('str')

In [ ]:
city_aggregate_features = df3.groupby(['Region_Code']).agg({'Age': ['mean', 'max', 'min','std'],
                                                           'Vintage': ['mean', 'max', 'min','std'],
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std','sum'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                     })
city_aggregate_features.columns = ['city_aggregate_features' + '_'.join(c).strip('_') for c in city_aggregate_features.columns]
df3 = pd.merge(df3, city_aggregate_features, on = ['Region_Code'], how='left')
city_aggregate_features1 = df3.groupby(['Region_Code','Vintage']).agg({'Age': ['mean', 'max', 'min','std'],
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std','sum'], 
                                                     'Occupation': ['nunique','count'],
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                     })
city_aggregate_features1.columns = ['city_aggregate_features1' + '_'.join(c).strip('_') for c in city_aggregate_features1.columns]
df3 = pd.merge(df3, city_aggregate_features1, on = ['Region_Code','Vintage'], how='left')

city_aggregate_features2 = df3.groupby(['Region_Code','Occupation']).agg({'Age': ['mean', 'max', 'min','std'],
                                                           'Vintage': ['mean', 'max', 'min','std'],
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std','sum'], 
                                        
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                     })
city_aggregate_features2.columns = ['city_aggregate_features2' + '_'.join(c).strip('_') for c in city_aggregate_features2.columns]
df3 = pd.merge(df3, city_aggregate_features2, on = ['Region_Code','Occupation'], how='left')

city_aggregate_features3 = df3.groupby(['Region_Code','Channel_Code']).agg({'Age': ['mean', 'max', 'min','std'],
                                                           'Vintage': ['mean', 'max', 'min','std'],
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std','sum'], 
                                        
                                                     'Occupation': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                     })
city_aggregate_features3.columns = ['city_aggregate_features3' + '_'.join(c).strip('_') for c in city_aggregate_features3.columns]
df3 = pd.merge(df3, city_aggregate_features3, on = ['Region_Code','Channel_Code'], how='left')

city_aggregate_features4 = df3.groupby(['Vintage','Occupation']).agg({'Age': ['mean', 'max', 'min','std'],
                                                           'Region_Code': ['mean', 'max', 'min','std'],
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std','sum'], 
                                              
                                                     'Channel_Code': ['nunique','count'] ,
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                     })
city_aggregate_features4.columns = ['city_aggregate_features4' + '_'.join(c).strip('_') for c in city_aggregate_features4.columns]
df3 = pd.merge(df3, city_aggregate_features4, on = ['Vintage','Occupation'], how='left')

city_aggregate_features5 = df3.groupby(['Vintage','Channel_Code']).agg({'Age': ['mean', 'max', 'min','std'],
                                                           'Region_Code': ['mean', 'max', 'min','std'],
                                                     'Avg_Account_Balance': ['mean', 'max', 'min','std','sum'], 
                                                     'Occupation': ['nunique','count'],
                
                                                     'Gender': ['nunique','count'] ,
                                                     'Credit_Product': ['nunique','count'] ,
                                                     'Is_Active': ['nunique','count'] ,
                                                     })
city_aggregate_features5.columns = ['city_aggregate_features5' + '_'.join(c).strip('_') for c in city_aggregate_features5.columns]
df3 = pd.merge(df3, city_aggregate_features5, on = ['Vintage','Channel_Code'], how='left')




In [ ]:
df3.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Age_Group,Total_premium_Channelwise,Mean_premium_Channelwise,Maximum_premium_Channelwise,Min_premium_Channelwise,Total_premium_regionwise,Mean_premium_regionwise,Max_premium_regionwise,Min_premium_regionwise,Age_groups_region_wise,regionwise_channels,Channelwise_regions,Unique_customers_based_Vinatge,Region_wise_Vehicle_Age_premium,Region_wise_Vehicle_Age_premium_mean,Region_wise_Vehicle_Age_premium_max,Channel_wise_Vehicle_Age_premium,Channel_wise_Vehicle_Age_premium_mean,Channel_wise_Vehicle_Age_premium_max,Rank_regionwise_premium,Rank_mean_regionwise_premium,Rank_max_regionwise_premium,Rank_min_regionwise_premium,Rank_regionwise_diff,Rank_channelwise_premium,Rank_mean_channelwise_premium,Rank_max_channelwise_premium,Rank_min_channelwise_premium,Rank_channelwise_diff,Rank_Channel_wise_Vehicle_Age_Premium,...,InsuranceGender1_13,InsuranceGender1_14,InsuranceGender2_0-99,InsuranceGender2_00,InsuranceGender2_01,InsuranceGender2_1-99,InsuranceGender2_10,InsuranceGender2_11,InsuranceGender3_00,InsuranceGender3_01,InsuranceGender3_02,InsuranceGender3_03,InsuranceGender3_10,InsuranceGender3_11,InsuranceGender3_12,InsuranceGender3_13,InsuranceGender4_01,InsuranceGender4_02,InsuranceGender4_03,InsuranceGender4_04,InsuranceGender4_11,InsuranceGender4_12,InsuranceGender4_13,InsuranceGender4_14,InsuranceGender5_01,InsuranceGender5_02,InsuranceGender5_03,InsuranceGender5_04,InsuranceGender5_11,InsuranceGender5_12,InsuranceGender5_13,InsuranceGender5_14,InsuranceGender5_21,InsuranceGender5_22,InsuranceGender5_23,InsuranceGender5_24,InsuranceGender5_31,InsuranceGender5_32,InsuranceGender5_33,InsuranceGender5_34
0,0,73,18,2,2,43,0,1045696,0,0.0,1,2852149051,1.241684e+06,7779267,95114,75114665124,1.471135e+06,10352009,45950,62,35,66,66,24910377386,1.540149e+06,10352009,49190672720,1.347985e+06,10352009,20313.0,20313.0,20313.0,20313.0,0.0,49671.0,49671.0,49671.0,49671.0,0.0,417.0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,30,27,1,1,32,0,581988,0,0.0,1,14068133361,1.034422e+06,9090894,56185,18069009345,9.808386e+05,9672677,79850,62,35,63,63,6200074070,9.664963e+05,9672677,90873818778,1.008477e+06,9701151,4968.0,4968.0,4968.0,4968.0,0.0,41357.0,41358.0,41359.0,41357.0,2.0,4223.0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,56,18,0,2,26,0,1484315,1,0.0,1,16092006404,1.024512e+06,9701151,76557,75114665124,1.471135e+06,10352009,45950,62,35,66,66,35292017443,1.400144e+06,8741290,64625208152,1.223406e+06,8680305,32226.0,32226.0,32226.0,32226.0,0.0,70668.0,70668.0,70668.0,70668.0,0.0,2296.0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,34,20,1,1,19,0,470454,0,0.0,1,17602048614,1.008309e+06,9307666,45950,7767689513,6.960295e+05,8086910,78818,62,35,63,63,3562632056,6.865739e+05,5222146,90873818778,1.008477e+06,9701151,2968.0,2968.0,2968.0,2968.0,0.0,24199.0,24199.5,24200.0,24199.0,1.0,742.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,30,32,1,1,33,0,886787,0,0.0,1,14212136644,1.040344e+06,9513952,22597,7839358912,9.599999e+05,8338925,50293,63,35,63,63,3017573796,9.712178e+05,7060467,90873818778,1.008477e+06,9701151,4124.0,4124.0,4124.0,4124.0,0.0,83294.0,83294.0,83294.0,83294.0,0.0,8316.0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
y=LabelEncoder()

In [ ]:
df3['Region_Code']=y.fit_transform(df3['Region_Code'])

In [ ]:
df3.drop('ID',axis=1,inplace=True)
#df3.drop('Credit_Product',axis=1,inplace=True)

In [ ]:
df3=combine_set

In [ ]:
df3 = pd.get_dummies(df3)

In [ ]:
train_df=df3[df3['Is_Lead'].isnull()==False]
test_df=df3[df3['Is_Lead'].isnull()==True]

X=train_df.drop(['Is_Lead'],axis=1)
y=train_df['Is_Lead'] 
X_main_test=test_df.drop(['Is_Lead'],axis=1)

In [ ]:
oof_pred               = np.zeros((len(train_df),))
y_pred_final           = np.zeros((len(test_df),))
num_models             = 3

n_splits               = 20
error                  = []

kf=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=294)
    
for i,(train_idx,val_idx) in enumerate(kf.split(X,y)):    
    
    wghts                     = [0]*num_models
    test_roc_score            = []
    
    
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], y.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))

    model1 = LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=-1,learning_rate=0.03,scale_pos_weight=7,objective='binary',metric='auc',
                 colsample_bytree=0.5,random_state=294,n_jobs=-1)
    model1.fit(X_train,y_train)
    testpred1 = model1.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred1))
    print("Test ROC AUC for model 1: %.4f"%(roc_auc_score(y_val, testpred1)))
    
    model2 = LGBMClassifier(boosting_type='gbdt',n_estimators=300,depth=-1,learning_rate=0.03,scale_pos_weight=7,objective='binary',metric='auc',
                 colsample_bytree=0.3,reg_alpha=2,random_state=294,n_jobs=-1)
    model2.fit(X_train,y_train)
    testpred2 = model2.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred2))
    print("Test ROC AUC for model 2: %.4f"%(roc_auc_score(y_val, testpred2)))
    
    model3 = LGBMClassifier(boosting_type='gbdt',n_estimators=400,depth=-1,learning_rate=0.03,scale_pos_weight=7,objective='binary',metric='auc',
                 colsample_bytree=0.4,reg_alpha=2,random_state=294,n_jobs=-1)
    model3.fit(X_train,y_train)
    testpred3 = model3.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred3))
    print("Test ROC AUC for model 3: %.4f"%(roc_auc_score(y_val, testpred3)))
    
    wghts              = np.exp(-1000*np.array(test_roc_score/sum(test_roc_score)))
    wghts              = wghts/sum(wghts)
    
    val_pred           = wghts[0]*testpred1+wghts[1]*testpred2 +wghts[2]*testpred3
    print('validation roc_auc_score fold-',i+1,': ',roc_auc_score(y_val, val_pred))
    
    oof_pred[val_idx]  = val_pred
    y_pred_final += (wghts[0]*model1.predict_proba(X_main_test)[:,1]+wghts[1]*model2.predict_proba(X_main_test)[:,1]+wghts[2]*model3.predict_proba(X_main_test)[:,1])/(n_splits)
    
    print('\n')
    
print('OOF ROC_AUC_Score:- ',(roc_auc_score(y,oof_pred)))


Fold: 1

Test ROC AUC for model 1: 0.8724
Test ROC AUC for model 2: 0.8713
Test ROC AUC for model 3: 0.8713
validation roc_auc_score fold- 1 :  0.8715299541936895



Fold: 2

Test ROC AUC for model 1: 0.8729
Test ROC AUC for model 2: 0.8727
Test ROC AUC for model 3: 0.8730
validation roc_auc_score fold- 2 :  0.8728885867834483



Fold: 3

Test ROC AUC for model 1: 0.8783
Test ROC AUC for model 2: 0.8791
Test ROC AUC for model 3: 0.8792
validation roc_auc_score fold- 3 :  0.8789041698603701



Fold: 4

Test ROC AUC for model 1: 0.8748
Test ROC AUC for model 2: 0.8747
Test ROC AUC for model 3: 0.8751
validation roc_auc_score fold- 4 :  0.8749439408947055



Fold: 5

Test ROC AUC for model 1: 0.8800
Test ROC AUC for model 2: 0.8798
Test ROC AUC for model 3: 0.8799
validation roc_auc_score fold- 5 :  0.8799755887578707



Fold: 6

Test ROC AUC for model 1: 0.8769
Test ROC AUC for model 2: 0.8763
Test ROC AUC for model 3: 0.8772
validation roc_auc_score fold- 6 :  0.8767825699648276



Fol

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351037 entries, 0 to 105311
Data columns (total 86 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Gender                                 351037 non-null  int64  
 1   Age                                    351037 non-null  int64  
 2   Region_Code                            351037 non-null  int64  
 3   Occupation                             351037 non-null  int64  
 4   Channel_Code                           351037 non-null  int64  
 5   Vintage                                351037 non-null  int64  
 6   Credit_Product                         351037 non-null  int64  
 7   Avg_Account_Balance                    351037 non-null  int64  
 8   Is_Active                              351037 non-null  int64  
 9   Is_Lead                                245725 non-null  float64
 10  Age_Group                              351037 non-null  

In [ ]:
df3.columns

Index(['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage',
       'Credit_Product', 'Avg_Account_Balance', 'Is_Active', 'Is_Lead',
       'Age_Group', 'Total_premium_Channelwise', 'Mean_premium_Channelwise',
       'Maximum_premium_Channelwise', 'Min_premium_Channelwise',
       'Total_premium_regionwise', 'Mean_premium_regionwise',
       'Max_premium_regionwise', 'Min_premium_regionwise',
       'Age_groups_region_wise', 'regionwise_channels', 'Channelwise_regions',
       'Unique_customers_based_Vinatge', 'Region_wise_Vehicle_Age_premium',
       'Region_wise_Vehicle_Age_premium_mean',
       'Region_wise_Vehicle_Age_premium_max',
       'Channel_wise_Vehicle_Age_premium',
       'Channel_wise_Vehicle_Age_premium_mean',
       'Channel_wise_Vehicle_Age_premium_max', 'Rank_regionwise_premium',
       'Rank_mean_regionwise_premium', 'Rank_max_regionwise_premium',
       'Rank_min_regionwise_premium', 'Rank_regionwise_diff',
       'Rank_channelwise_premium', 'Rank

In [ ]:
cat_col=['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage',
       'Credit_Product', 'Avg_Account_Balance', 'Is_Active',
       'Age_Group', 'Total_premium_Channelwise',
       'Maximum_premium_Channelwise', 'Min_premium_Channelwise',
       'Total_premium_regionwise',
       'Max_premium_regionwise', 'Min_premium_regionwise',
       'Age_groups_region_wise', 'regionwise_channels', 'Channelwise_regions',
       'Unique_customers_based_Vinatge', 'Region_wise_Vehicle_Age_premium',
      
       'Region_wise_Vehicle_Age_premium_max',
       'Channel_wise_Vehicle_Age_premium',
      
       'Channel_wise_Vehicle_Age_premium_max',
       'InsuranceLicense_0-99', 'InsuranceLicense_00', 'InsuranceLicense_01',
       'InsuranceLicense_1-99', 'InsuranceLicense_10', 'InsuranceLicense_11',
       'InsuranceGender_00', 'InsuranceGender_01', 'InsuranceGender_02',
       'InsuranceGender_03', 'InsuranceGender_10', 'InsuranceGender_11',
       'InsuranceGender_12', 'InsuranceGender_13', 'InsuranceGender1_01',
       'InsuranceGender1_02', 'InsuranceGender1_03', 'InsuranceGender1_04',
       'InsuranceGender1_11', 'InsuranceGender1_12', 'InsuranceGender1_13',
       'InsuranceGender1_14', 'InsuranceGender2_0-99', 'InsuranceGender2_00',
       'InsuranceGender2_01', 'InsuranceGender2_1-99', 'InsuranceGender2_10',
       'InsuranceGender2_11', 'InsuranceGender3_00', 'InsuranceGender3_01',
       'InsuranceGender3_02', 'InsuranceGender3_03', 'InsuranceGender3_10',
       'InsuranceGender3_11', 'InsuranceGender3_12', 'InsuranceGender3_13',
       'InsuranceGender4_01', 'InsuranceGender4_02', 'InsuranceGender4_03',
       'InsuranceGender4_04', 'InsuranceGender4_11', 'InsuranceGender4_12',
       'InsuranceGender4_13', 'InsuranceGender4_14']

In [ ]:
train = df3[df3['Is_Lead'].isnull()!= True]
test = df3[df3['Is_Lead'].isnull()== True]

In [ ]:
test.drop(['Is_Lead'],axis=1,inplace=True)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
X = train.drop(["Is_Lead"], axis=1)
Y = train["Is_Lead"]

In [ ]:
#train_x,test_x,train_y,test_y=train_test_split(df,df1,test_size=0.25,random_state=1,shuffle=True)
oof_pred               = np.zeros((len(train),))
y_pred_final           = np.zeros((len(test),))
num_models             = 3

n_splits               = 2
error                  = []

kf=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=294)
    
for i,(train_idx,val_idx) in enumerate(kf.split(X,Y)):    
    
    wghts                     = [0]*num_models
    test_roc_score            = []
    
    
    X_train, y_train = X.iloc[train_idx,:], Y.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], Y.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))

    model1 = cat.CatBoostClassifier(learning_rate = 0.03,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])
    model1.fit(X_train,y_train,cat_features=cat_col,eval_set=(X_val, y_val),early_stopping_rounds=30,verbose=100)
    testpred1 = model1.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred1))
    print("Test ROC AUC for model 1: %.4f"%(roc_auc_score(y_val, testpred1)))
    
    model2 = cat.CatBoostClassifier(learning_rate = 0.04,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])
    model2.fit(X_train,y_train,cat_features=cat_col,eval_set=(X_val, y_val),early_stopping_rounds=40,verbose=100)
    testpred2 = model2.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred2))
    print("Test ROC AUC for model 2: %.4f"%(roc_auc_score(y_val, testpred2)))
    
    model3 = cat.CatBoostClassifier(learning_rate = 0.05,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])
    model3.fit(X_train,y_train,cat_features=cat_col,eval_set=(X_val, y_val),early_stopping_rounds=20,verbose=100)
    testpred3 = model3.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred3))
    print("Test ROC AUC for model 3: %.4f"%(roc_auc_score(y_val, testpred3)))
    
    wghts              = np.exp(-1000*np.array(test_roc_score/sum(test_roc_score)))
    wghts              = wghts/sum(wghts)
    
    val_pred           = wghts[0]*testpred1+wghts[1]*testpred2 +wghts[2]*testpred3
    print('validation roc_auc_score fold-',i+1,': ',roc_auc_score(y_val, val_pred))
    
    oof_pred[val_idx]  = val_pred
    y_pred_final += (wghts[0]*model1.predict_proba(test)[:,1]+wghts[1]*model2.predict_proba(test)[:,1]+wghts[2]*model3.predict_proba(test)[:,1])/(n_splits)
    
    print('\n')
    
print('OOF ROC_AUC_Score:- ',(roc_auc_score(Y,oof_pred)))


Fold: 1

0:	learn: 0.6731169	test: 0.6728809	best: 0.6728809 (0)	total: 1.01s	remaining: 16m 46s
100:	learn: 0.4089662	test: 0.4075632	best: 0.4075632 (100)	total: 1m 13s	remaining: 10m 55s
200:	learn: 0.4047124	test: 0.4044949	best: 0.4044949 (200)	total: 2m 36s	remaining: 10m 21s
300:	learn: 0.4029850	test: 0.4039242	best: 0.4039242 (299)	total: 3m 51s	remaining: 8m 57s
400:	learn: 0.4014316	test: 0.4035404	best: 0.4035404 (400)	total: 5m 7s	remaining: 7m 39s
500:	learn: 0.3994814	test: 0.4033659	best: 0.4033659 (500)	total: 6m 28s	remaining: 6m 27s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.403347371
bestIteration = 517

Shrink model to first 518 iterations.
Test ROC AUC for model 1: 0.8743
0:	learn: 0.6666592	test: 0.6663480	best: 0.6663480 (0)	total: 885ms	remaining: 14m 43s
100:	learn: 0.4069097	test: 0.4059249	best: 0.4059249 (100)	total: 1m 16s	remaining: 11m 19s
200:	learn: 0.4035630	test: 0.4040277	best: 0.4040277 (200)	total: 2m 31s	remaining: 10m 1

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=0.25,random_state=1,shuffle=True)

In [ ]:
model=GaussianNB()
model.fit(train_x,train_y)
o=model.predict(test_x)
print(accuracy_score(test_y,o))


0.726478057038677


In [ ]:
!pip install catboost 


In [ ]:
import catboost as cat

In [ ]:
m1=y_pred_final

In [ ]:
pre=pre*0.5 +df3['0']*0.5
pre

0         0.223095
1         0.973336
2         0.281121
3         0.141629
4         0.136330
            ...   
105307    0.995142
105308    0.891831
105309    0.339192
105310    0.680837
105311    0.264482
Name: 0, Length: 105312, dtype: float64

In [ ]:
y_pred_final

array([0.22109801, 0.97335096, 0.27968477, ..., 0.32779139, 0.67609816,
       0.26583231])

In [ ]:
df3=pd.read_csv('/content/doccc20.csv')
df3.head()

,Unnamed: 0,0
0,0,0.223095
1,1,0.973336
2,2,0.281121
3,3,0.141629
4,4,0.136330


In [ ]:
df3.drop(['Unnamed: 0'],axis=1,inplace=True)


In [ ]:
pre=model4.predict_proba(test)[:,1]
pre

array([0.05387029, 0.844847  , 0.05202898, ..., 0.08730729, 0.21979302,
       0.04923308], dtype=float32)

In [ ]:
df4=pd.DataFrame(pre)
df4

,0
0,0.223095
1,0.973336
2,0.281121
3,0.141629
4,0.136330
...,...
105307,0.995142
105308,0.891831
105309,0.339192
105310,0.680837


In [ ]:
df4.to_csv('doccc20.csv')

In [ ]:
def frequency_encoding(column_name,output_column_name,df):
    fe_pol = (df.groupby(column_name).size()) / len(df)
    df[output_column_name] = df[column_name].apply(lambda x : fe_pol[x])


def feature_engineering(df):
    le = LabelEncoder()
    
     #Interaction Feature (Combining 2 categorical features and performing frequency encoding)
        
    cat_features=[]
    le_features=[]
    columns=['City_Code','Accomodation_Type','Reco_Insurance_Type','Health Indicator','Is_Spouse','Region_Code','Holding_Policy_Type','Reco_Policy_Cat']

    comb = combinations(columns, 2) 

    for i in list(comb):  
        df[f'{i[0]}_{i[1]}']=df[i[0]].astype(str)+'_'+df[i[1]].astype(str)
        df[f'{i[0]}_{i[1]}_le']=le.fit_transform(df[f'{i[0]}_{i[1]}'])
        le_features.append(f'{i[0]}_{i[1]}_le')
        frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',df)
        cat_features.append(f'{i[0]}_{i[1]}')   
        
    #Frequency Encoding
    
    frequency_encoding('Region_Code','Region_Code_fe',df)
    frequency_encoding('Reco_Policy_Cat','Reco_Policy_Cat_fe',df)
    
    #Deriving characteristics of each city by creating aggregate features
    
    city_aggregate_features = df.groupby(['City_Code']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'Reco_Policy_Cat': ['nunique','count'] ,
                                                     })
    city_aggregate_features.columns = ['city_aggregate_features' + '_'.join(c).strip('_') for c in city_aggregate_features.columns]
    df = pd.merge(df, city_aggregate_features, on = ['City_Code'], how='left')

 
    city_region_aggregate_features = df.groupby(['City_Code','Region_Code']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'],  
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'Reco_Policy_Cat': ['nunique','count'] ,
                                                     })
    city_region_aggregate_features.columns = ['city_region_aggregate_features' + '_'.join(c).strip('_') for c in city_region_aggregate_features.columns]
    df = pd.merge(df, city_region_aggregate_features, on = ['City_Code','Region_Code'], how='left')

    city_recopolicycat_aggregate_features = df.groupby(['City_Code','Reco_Policy_Cat']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] 
                                                     })
    city_recopolicycat_aggregate_features.columns = ['city_recopolicycat_aggregate_features' + '_'.join(c).strip('_') for c in city_recopolicycat_aggregate_features.columns]
    df = pd.merge(df, city_recopolicycat_aggregate_features, on = ['City_Code','Reco_Policy_Cat'], how='left')
    
    city_regioncoderecopolicycat_aggregate_features = df.groupby(['City_Code','Region_Code_Reco_Policy_Cat']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'Reco_Policy_Cat': ['nunique','count'] ,
                                                     })

    city_regioncoderecopolicycat_aggregate_features.columns = ['city_regioncoderecopolicycat_aggregate_features' + '_'.join(c).strip('_') for c in city_regioncoderecopolicycat_aggregate_features.columns]
    df = pd.merge(df, city_regioncoderecopolicycat_aggregate_features, on = ['City_Code','Region_Code_Reco_Policy_Cat'], how='left')
    
    for i in cat_features:
        df[f'city_{i}_max']=df.groupby('City_Code')[i].transform('max')
        df[f'city_{i}_min']=df.groupby('City_Code')[i].transform('min')
        df[f'city_{i}_mean']=df.groupby('City_Code')[i].transform('mean')
        df[f'city_{i}_std']=df.groupby('City_Code')[i].transform('std')

    
        df[f'city_region_{i}_max']=df.groupby(['City_Code','Region_Code'])[i].transform('max')
        df[f'city_region_{i}_min']=df.groupby(['City_Code','Region_Code'])[i].transform('min')
        df[f'city_region_{i}_mean']=df.groupby(['City_Code','Region_Code'])[i].transform('mean')
        df[f'city_region_{i}_std']=df.groupby(['City_Code','Region_Code'])[i].transform('std')

    
        df[f'city_recopolicycat_{i}_max']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('max')
        df[f'city_recopolicycat_{i}_min']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('min')
        df[f'city_recopolicycat_{i}_mean']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('mean')
        df[f'city_recopolicycat_{i}_std']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform('std')
        
    
    #features on reco_policy_cat
    
    recopolicycat_aggregate_features = df.groupby(['Reco_Policy_Cat']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std','sum'],   
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique'],
                                                     'Reco_Insurance_Type': ['nunique'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'City_Code': ['nunique','count'] ,
                                                     })
    recopolicycat_aggregate_features.columns = ['recopolicycat_aggregate_features' + '_'.join(c).strip('_') for c in recopolicycat_aggregate_features.columns]
    df = pd.merge(df, recopolicycat_aggregate_features, on = ['Reco_Policy_Cat'], how='left')
        
        

    #features on Holding_Policy_Type 
    
    holdingpolicytype_aggregate_features = df.groupby(['Holding_Policy_Type']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std'], 
                                                     'Region_Code': ['nunique','count'], 
                                                     'Accomodation_Type': ['nunique','count'],
                                                     'Reco_Insurance_Type': ['nunique','count'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'City_Code': ['nunique','count'] ,
                                                     })
    holdingpolicytype_aggregate_features.columns = ['holdingpolicytype_aggregate_features' + '_'.join(c).strip('_') for c in holdingpolicytype_aggregate_features.columns]
    df = pd.merge(df, holdingpolicytype_aggregate_features, on = ['Holding_Policy_Type'], how='left')
    
    #Deriving characteristics of Accomodation_Type by creating aggregate features
    
    Accomodation_Type_aggregate_features = df.groupby(['Accomodation_Type']).agg({'Lower_Age': ['mean', 'max', 'min','std'],
                                                     'Reco_Policy_Premium': ['mean', 'max', 'min','std','sum'],   
                                                     'Region_Code': ['nunique','count'], 
                                                     'Reco_Insurance_Type': ['nunique'] ,
                                                     'Health Indicator': ['nunique','count'] ,
                                                     'Holding_Policy_Type': ['nunique','count'] ,
                                                     'City_Code': ['nunique','count'] ,
                                                     })
    Accomodation_Type_aggregate_features.columns = ['Accomodation_Type_aggregate_features' + '_'.join(c).strip('_') for c in Accomodation_Type_aggregate_features.columns]
    df = pd.merge(df, Accomodation_Type_aggregate_features, on = ['Accomodation_Type'], how='left')
    
    #Deriving characteristics of Interaction_features by creating aggregate features (These interaction feature are selected for aggregating based on its feature importance)
    
    Region_CodeReco_Policy_Cat_grpd = df.groupby(['Region_Code_Reco_Policy_Cat']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    Region_CodeReco_Policy_Cat_grpd.columns = ['grpd_by_Region_Code_Reco_Policy_Cat_' + '_'.join(c).strip('_') for c in Region_CodeReco_Policy_Cat_grpd.columns]
    df = pd.merge(df, Region_CodeReco_Policy_Cat_grpd, on = ['Region_Code_Reco_Policy_Cat'], how='left')


    City_CodeRegion_Code_grpd = df.groupby(['City_Code_Region_Code']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    City_CodeRegion_Code_grpd.columns = ['grpd_by_City_CodeRegion_Code_' + '_'.join(c).strip('_') for c in City_CodeRegion_Code_grpd.columns]
    df = pd.merge(df, City_CodeRegion_Code_grpd, on = ['City_Code_Region_Code'], how='left')


    City_CodeReco_Policy_Cat_grpd = df.groupby(['City_Code_Reco_Policy_Cat']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    City_CodeReco_Policy_Cat_grpd.columns = ['grpd_by_City_CodeReco_Policy_Cat_' + '_'.join(c).strip('_') for c in City_CodeReco_Policy_Cat_grpd.columns]
    df = pd.merge(df, City_CodeReco_Policy_Cat_grpd, on = ['City_Code_Reco_Policy_Cat'], how='left')


    Holding_Policy_TypeReco_Policy_Cat_grpd = df.groupby(['Holding_Policy_Type_Reco_Policy_Cat']).agg({ 'Reco_Policy_Premium': ['mean', 'max', 'min', 'std']})                                                              
                                                     
    Holding_Policy_TypeReco_Policy_Cat_grpd.columns = ['grpd_by_Holding_Policy_TypeReco_Policy_Cat_' + '_'.join(c).strip('_') for c in Holding_Policy_TypeReco_Policy_Cat_grpd.columns]
    df = pd.merge(df, Holding_Policy_TypeReco_Policy_Cat_grpd, on = ['Holding_Policy_Type_Reco_Policy_Cat'], how='left')
    
    return df,le_features